# 서울특별시 다산콜센터(☎120)의 주요 민원
* "120주요질문"은 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보이다.
* https://opengov.seoul.go.kr/civilappeal/list

In [1]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용
from bs4 import BeautifulSoup as bs
import time
# 대량 데이터 처리시 진행 상황을 표시
from tqdm import tqdm, trange
# 정규표현식
import re

In [13]:
# 120 다산 콜센터의 첫 페이지를 먼저 불러와 크롤링할 내용을 본다.
base_url = "https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&"

In [41]:
table = pd.read_html(base_url, encoding="utf8")[0]
table

,번호,제목,생산일,조회수
0,2669,아빠 육아휴직 장려금,2021-08-17,13
1,2668,[서울산업진흥원] 서울메이드란?,2021-06-29,104
2,2667,(강북구) 정비중,2021-06-02,164
3,2666,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,203
4,2665,마포 뇌병변장애인 비전센터,2021-03-12,378
5,2664,2021년도 중1·고1 신입생 입학준비금 지원,2021-02-09,228
6,2663,[동대문구]반려견 놀이터,2020-12-08,585
7,2662,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,703
8,2661,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,453
9,2660,서울시 '우리동네꼼꼼육아정보' 책자,2019-10-23,1128


In [68]:
# 웹페이지의 결과를 받아온다.
response = requests.get( base_url )
# html 태그를 파싱해 올 수 있도록 한다.
html = bs(response.text, 'html.parser')

In [69]:
response.status_code

200

In [43]:
a_list = html.select("td > a")
a_link_no = [a["href"].replace("/civilappeal/", "") for a in a_list]

In [46]:
table["view_no"] = a_link_no
table

,번호,제목,생산일,조회수,no,view_no
0,2669,아빠 육아휴직 장려금,2021-08-17,13,23522464,23522464
1,2668,[서울산업진흥원] 서울메이드란?,2021-06-29,104,23194045,23194045
2,2667,(강북구) 정비중,2021-06-02,164,23032485,23032485
3,2666,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,203,22904492,22904492
4,2665,마포 뇌병변장애인 비전센터,2021-03-12,378,22477798,22477798
5,2664,2021년도 중1·고1 신입생 입학준비금 지원,2021-02-09,228,22227896,22227896
6,2663,[동대문구]반려견 놀이터,2020-12-08,585,21780668,21780668
7,2662,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,703,21212235,21212235
8,2661,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,453,21095911,21095911
9,2660,서울시 '우리동네꼼꼼육아정보' 책자,2019-10-23,1128,18955676,18955676


In [107]:
def get_one_page(page_no):
    base_url = f"https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&page={page_no}"
    table = pd.read_html(base_url, encoding="utf8")[0]
    # 웹페이지의 결과를 받아온다.
    response = requests.get( base_url )
    # html 태그를 파싱해 올 수 있도록 한다.
    try:
        html = bs(response.text, 'html.parser')
        a_list = html.select("td > a")
        a_link_no = [a["href"].replace("/civilappeal/", "") for a in a_list]
        table["view_no"] = a_link_no
        return table
    except:
        return pd.DataFrame()

In [108]:
get_one_page(10000)

""


In [110]:
if table.empty:
    print(table)

In [111]:
page_no = 1
table_list = []
while True:
    table = get_one_page(page_no)
    table_list.append(table)
    page_no = page_no + 1
    time.sleep(0.01)
    if table.empty:
        break

In [112]:
df = pd.concat(table_list)

In [113]:
df.to_csv("seoul-120-list.csv", index=False)

In [114]:
pd.read_csv("seoul-120-list.csv")

,번호,제목,생산일,조회수,view_no
0,2669,아빠 육아휴직 장려금,2021-08-17,13,23522464
1,2668,[서울산업진흥원] 서울메이드란?,2021-06-29,104,23194045
2,2667,(강북구) 정비중,2021-06-02,164,23032485
3,2666,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,203,22904492
4,2665,마포 뇌병변장애인 비전센터,2021-03-12,378,22477798
...,...,...,...,...,...
2664,5,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,2007-01-07,251,2894353
2665,4,수돗물의 톤당 원가는?,2007-01-07,220,19353862
2666,3,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,2007-01-07,188,19699584
2667,2,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,2007-01-07,167,2898293
